In [62]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [86]:
data = pd.read_csv('umn_foursquare_datasets/checkins.dat', sep='|', skipinitialspace=True)
data.columns = data.columns.map(str.strip)
data = data.iloc[:-1]
data = data[data['latitude'].notna()]
data = data.iloc[:100000]

/Users/s.komissarov/opt/anaconda3/envs/test/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
from sklearn.cluster import MeanShift

In [89]:
%%time
ms = MeanShift(bandwidth=0.1)
clusters = ms.fit_predict(data[['latitude', 'longitude']])

CPU times: user 3min 6s, sys: 1.88 s, total: 3min 8s
Wall time: 3min 10s


In [118]:
offices = {
    'Los Angeles' : [33.751277, -118.188740],
    'Miami' : [25.867736, -80.324116],
    'London' : [51.503016, -0.075479],
    'Amsterdam' : [52.378894, 4.885084],
    'Beijing' : [39.366487, 117.036146],
    'Sydney' : [-33.868457, 151.205134]
}

In [95]:
clusters_ = pd.Series(clusters)

In [107]:
clusters_.value_counts()[clusters_.value_counts() > 15].index

Int64Index([  0,   1,   2,   4,   3,   5,   6,   7,   8,   9,
            ...
            575, 578, 581, 593, 572, 590, 397, 579, 450, 573],
           dtype='int64', length=592)

In [113]:
centres = ms.cluster_centers_[clusters_.value_counts()[clusters_.value_counts() > 15].index]

In [117]:
point = centres[0] 

In [123]:
def distance(point):
    dst = []
    for p in offices.values():
        dst.append(np.sqrt(np.sum((point - p) ** 2)))

    return pd.Series(dst, index=offices.keys())

In [126]:
centroids = pd.DataFrame(centres, columns=['lat', 'lon'])

In [129]:
dst = centroids.apply(distance, axis=1)

In [131]:
dst[['lat', 'lon']] = centroids[['lat', 'lon']]

In [137]:
dst['min_dst'] = dst.iloc[:, :6].min(axis=1)

In [139]:
dst.sort_values(by='min_dst')

,Los Angeles,Miami,London,Amsterdam,Beijing,Sydney,lat,lon,min_dst
397,277.748513,239.108983,173.702814,169.843186,80.806596,0.007835,-33.860630,151.204776,0.007835
366,124.481781,89.243306,5.043393,0.009353,112.895557,169.838226,52.372964,4.892317,0.009353
406,38.686226,0.022674,84.245501,89.239728,197.817651,239.100714,25.845672,-80.318891,0.022674
57,119.390298,84.196071,0.050058,5.086603,117.788602,173.750558,51.502991,-0.125537,0.050058
52,0.070848,38.649627,119.391731,124.427390,235.250702,277.726422,33.809878,-118.148924,0.070848
...,...,...,...,...,...,...,...,...,...
105,91.667105,59.806393,88.323109,91.759329,175.350282,198.112273,-23.549518,-46.638219,59.806393
413,82.342126,60.090305,110.374252,114.262928,201.233824,221.776316,-33.425741,-70.570740,60.090305
249,93.976814,61.287238,86.005155,89.333880,171.919316,194.723684,-22.903539,-43.209587,61.287238
101,158.414912,119.741616,63.428113,61.450391,89.754665,119.493762,-0.301226,36.522964,61.450391
